In [ ]:
 !pip install pandas-datareader
 !pip install pmdarima

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas_datareader as pdr
import pandas as pd
import numpy as np
import math
from datetime import datetime
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
#from numpy import sqrt 
from sklearn.metrics import mean_squared_error
#from math import sqrt
from pandas import to_datetime
from prophet import Prophet
from matplotlib import pyplot
from pandas import DataFrame
from sklearn.model_selection import train_test_split
#from matplotlib import pyplot


def parser(s):
    return datetime.strptime(s, '%Y-%m-%d')

df = pd.read_csv('LLOYDS_Monthly.csv', parse_dates=[0], index_col=0, squeeze=True, date_parser=parser)

df.drop(df.columns.difference(['Date','Close']), 1, inplace=True)
df.dropna(inplace = True)


#Train Test Split

train_size = 0.8
split_idx = round(len(df)* train_size)

train = df.iloc[:split_idx]
test = df.iloc[split_idx:]

pred_start_date = test.index[0]
#print(pred_start_date)
pred_end_date = test.index[-1]
#print(pred_end_date)

#ARIMA
df = df.iloc[1: , :]
model = sm.tsa.arima.ARIMA(train,order=(1,0,1))
model_fit = model.fit()
modelvalues = model_fit.predict(start=1, end=len(df))
df['Predicted_ARIMA'] = modelvalues.values
predicted_arima = df['Predicted_ARIMA']
str_msg = ""
str_msg = str_msg + model_fit.summary().as_text() 
# #df[['Close','Predicted_ARIMA']].plot(figsize=(12,8), title = 'ARIMA')
# #print(df)

from sklearn.metrics import r2_score 
a =df['Close'] 
b =df['Predicted_ARIMA']
R_square = r2_score(a, b) 
print('Coefficient of Determination for ARIMA Model', R_square) 

#Auto ARIMA

#df.drop(df.columns.difference(['Close']), 1, inplace=True)
model = auto_arima(train, start_p=0, start_q=0)
forecast = model.predict(n_periods=len(df))
forecast = pd.DataFrame(forecast,index = df.index,columns=['Prediction'])
df['Prediction'] = forecast.values
str_msg1 = ""
str_msg1 = str_msg1 + model.summary().as_text()

from sklearn.metrics import r2_score 
a =df['Close'] 
b =df['Prediction']
R_square = r2_score(a, b) 
print('Coefficient of Determination for AUTO ARIMA Model', R_square)

#SARIMA

model_SARIMA=SARIMAX(train,order=(2,1,0),seasonal_order=(1,1,0,12))
model_SARIMA_fit=model_SARIMA.fit()
modelvalues1 = model_SARIMA_fit.predict(start=1,end=len(df))
df['Predicted_SARIMA']=modelvalues1.values
predicted_sarima = df['Predicted_SARIMA']
residuals=df['Close']-df['Predicted_SARIMA']
str_msg2 = ""
str_msg2 = str_msg2 + model_SARIMA_fit.summary().as_text()

from sklearn.metrics import r2_score 
a =df['Close'] 
b =df['Predicted_SARIMA']
R_square = r2_score(a, b) 
print('Coefficient of Determination for SARIMA Model', R_square)

#FB PROPHET

df1 = pd.read_csv('LLOYDS_Monthly.csv')
df1.drop(df1.columns.difference(['Date','Close']), 1, inplace=True)
df1.columns = ['ds', 'y']
df1['ds']= to_datetime(df1['ds'])

train_size = 0.8
split_idx = round(len(df1)* train_size)

train1 = df1.iloc[:split_idx]
test1 = df1.iloc[split_idx:]

# define the model
model = Prophet(daily_seasonality=True, weekly_seasonality=True)
# fit the model
model.fit(df1)
forecast_3 = model.predict(test1)
df_past = pd.DataFrame(forecast_3)
# define the period for which we want a prediction
future = list()
for i in range(3,13):
    date = '2022-%02d' % i
    future.append([date])
future = DataFrame(future)
future.columns = ['ds']
future['ds']= to_datetime(future['ds'])
# use the model to make a forecast
forecast_4 = model.predict(future)
df_future = pd.DataFrame(forecast_4)
# summarize the forecast
#print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])






# #Plot
# fig, axs = plt.subplots(2, 2, figsize=(16,7))
# axs[0, 0].plot(train, label='Train')
# axs[0, 0].plot(test, label='Test')
# axs[0, 0].plot(predicted_arima, label='Prediction')
# #axs[0,0].fill_between(lower_series.index, lower_series, upper_series, color='k', alpha=.10)
# axs[0, 0].legend(loc = 'Best')
# axs[0, 0].set_title("ARIMA")
# axs[0, 1].plot(train, label='Train')
# axs[0, 1].plot(test, label='Test')
# axs[0, 1].plot(forecast, label='Prediction')
# #axs[0,1].fill_between(lower_series.index, lower_series, upper_series, color='k', alpha=.10)
# axs[0, 1].legend(loc = 'Best')
# axs[0, 1].set_title("AUTO ARIMA")
# axs[1, 0].plot(train, label='Train')
# axs[1, 0].plot(test, label='Test')
# axs[1, 0].plot(predicted_sarima, label='Prediction')
# #axs[1,0].fill_between(lower_series.index, lower_series, upper_series, color='k', alpha=.10)
# axs[1, 0].legend(loc = 'Best')
# axs[1, 0].set_title("SARIMA")
# model.plot(forecast_3)
# #model.plot(forecast_4)
# plt.show()

df.reset_index(inplace=True)
#print(df)


/home/vamshi/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
/home/vamshi/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
/home/vamshi/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)


Coefficient of Determination for ARIMA Model 0.6406758268428863
Coefficient of Determination for AUTO ARIMA Model -0.001953593860240632


/home/vamshi/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)
/home/vamshi/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  % freq, ValueWarning)


Coefficient of Determination for SARIMA Model 0.6838472309631527


In [ ]:
!pip install dash pandas
!pip install dash==2.0.0
!pip install -q jupyter_dash==0.3.0
!pip install -q dash-cytoscape

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
from dash.dependencies import Input, Output

import plotly.tools as tls
from plotly.subplots import make_subplots

def parser(s):
    return datetime.strptime(s, '%Y-%m-%d')

app = dash.Dash()



app.layout =  html.Div(style={'backgroundColor':'#070d30'}, children=[
    html.Div([
                 html.H1("TIME SERIES ANALYSIS OF DATA", 
                          style={'textAlign': 'center', 'marginTop': '20px', 'marginBottom': '20px',
                                 'color':'white'}, 
                          className="text-center", id="heading")
                ],className="row"),

        

         html.Div(children=[
                            
                  dcc.Upload(id='upload-data',
                              children=html.Div(['Drag and Drop or ',html.A('Select Files')]),
                              style={'width': '98%','height': '60px',
                                     'lineHeight': '60px',
                                     'borderWidth': '4px',
                                     'borderStyle': 'dashed',
                                     'borderRadius': '5px',
                                     'textAlign': 'center',
                                     'margin': '10px',
                                     'color':'white'},
                              multiple=False),          
                           
                  html.Hr(),
                  html.H4("Outputs for ARIMA MODEL", 
                          style={'textAlign': 'center', 'marginTop': '20px', 'marginBottom': '20px',
                                 'color':'white'}, 
                          className="text-center", id="heading1"),
                  

                  dcc.Graph(id='line-graph', style={'display': 'inline-block', 'width':'50%'},
                  figure=go.Figure(data = [go.Scatter(
                                              x = df['Date'],
                                              y = df['Close'],
                                              mode = 'lines',
                                              name = 'Observed',
                                              line={'color': 'orange'}
                                              ),
                                   go.Scatter(
                                              x = df['Date'],
                                              y = df['Predicted_ARIMA'],
                                              mode = 'lines',
                                              name = 'Predicted',
                                              line={'color': 'green'}),],
                                    layout=go.Layout(title={'text':'Close prices Prediction for ARIMA model',
                                                            'y':0.9,
                                                            'x':0.5,
                                                            'xanchor': 'center',
                                                            'yanchor': 'top'},
                                                     #plot_bgcolor='#070d30',
                                                     xaxis_title="Date", 
                                                     yaxis_title="Closing price in pounds (£)")
                                    ), className="Three columns"),
                            
                  
                  dcc.Textarea(id='run-model-echo', readOnly='readonly', placeholder='',
                                value= str_msg,
                                rows='40',
                                style={'display': 'inline', 'width': '45%', 'height':'1000', 'background-color': '#F5F5F5', 
                                       'opacity': 0.5, 'fontSize': 10, 'draggable': False, 'text-align':'center'}, 
                               className="three columns"),
            

                  html.Hr(),
                  html.H4("Outputs for AUTO ARIMA MODEL", 
                          style={'textAlign': 'center', 'marginTop': '20px', 'marginBottom': '20px',
                                 'color':'white'}, 
                          className="text-center", id="heading2"),


            dcc.Graph(id='line-graph1', style={'display': 'inline-block', 'width':'50%'},
            figure=go.Figure(data = [go.Scatter(
                                              x = df['Date'],
                                              y = df['Close'],
                                              mode = 'lines',
                                              name = 'Observed',
                                              line={'color': 'orange'}),
                                   go.Scatter(
                                              x = df['Date'],
                                              y = df['Prediction'],
                                              mode = 'lines',
                                              name = 'Predicted',
                                              line={'color': 'green'}),],
                                    layout=go.Layout(title={'text':'Close prices Prediction for AUTO ARIMA model',
                                                            'y':0.9,
                                                            'x':0.5,
                                                            'xanchor': 'center',
                                                            'yanchor': 'top'},
                                                     xaxis_title="Date", 
                                                     yaxis_title="Closing price in pounds (£)")
                                    ), className="Three columns"),
                            

          dcc.Textarea(id='run-model-echo1', readOnly='readonly', placeholder='',
                                value= str_msg1,
                                rows='40',
                                style={'display': 'inline', 'width': '45%', 'height':'1000', 'background-color': '#F5F5F5', 
                                       'opacity': 0.5, 'fontSize': 10, 'draggable': False, 'text-align':'center'}, 
                               className="three columns"),
                                     
         
                  html.Hr(),
                  html.H4("Outputs for SARIMA MODEL", 
                          style={'textAlign': 'center', 'marginTop': '20px', 'marginBottom': '20px',
                                 'color':'white'}, 
                          className="text-center", id="heading3"),


          dcc.Graph(id='line-graph2', style={'display': 'inline-block','width':'50%'},
          figure=go.Figure(data = [go.Scatter(
                                              x = df['Date'],
                                              y = df['Close'],
                                              mode = 'lines',
                                              name = 'Observed',
                                              line={'color': 'orange'}),
                                   go.Scatter(
                                              x = df['Date'],
                                              y = df['Predicted_SARIMA'],
                                              mode = 'lines',
                                              name = 'Predicted',
                                              line={'color': 'green'}),],
                                    layout=go.Layout(title={'text':'Close prices Prediction for SARIMA model',
                                                            'y':0.9,
                                                            'x':0.5,
                                                            'xanchor': 'center',
                                                            'yanchor': 'top'},
                                                     xaxis_title="Date", 
                                                     yaxis_title="Closing price in pounds (£)")
                                    ), className="six columns"),
                   
          
          dcc.Textarea(id='run-model-echo2', readOnly='readonly', placeholder='',
                                value= str_msg2,
                                rows='40',
                                style={'display': 'inline', 'width': '45%', 'height':'1000', 'background-color': '#F5F5F5', 
                                       'opacity': 0.5, 'fontSize': 10, 'draggable': False, 'text-align':'center'}, 
                               className="three columns"),
        

                  html.Hr(),
                  html.H4("Outputs for FB Prophet MODEL", 
                          style={'textAlign': 'center', 'marginTop': '20px', 'marginBottom': '20px',
                                 'color':'white'}, 
                          className="text-center", id="heading4"),


          dcc.Graph(id='line-graph3', style={'display': 'inline-block', 'width':'50%'},
          figure=go.Figure(data = [go.Scatter(
                                              x = df['Date'],
                                              y = df['Close'],
                                              mode = 'lines',
                                              name = 'Observed',
                                              line={'color': 'orange'}),
                                   go.Scatter(
                                              x = df_past['ds'],
                                              y = df_past['yhat'],
                                              mode = 'lines',
                                              name = 'Predicted',
                                              line={'color': 'green'}),
                                                      

                                   go.Scatter(
                                              x = df_past['ds'],
                                              y = df_past['yhat_lower'],
                                              mode = 'lines',
                                              fill = None,
                                              name = 'Lower range',
                                              line={'color': 'blue'}),
                                   go.Scatter(
                                              x = df_past['ds'],
                                              y = df_past['yhat_upper'],
                                              mode = 'lines',
                                              fill = 'tonexty',
                                              name = 'Upper range',
                                              line={'color': 'blue'}),
                    ],
                                    layout=go.Layout(title={'text':'Close prices Prediction for FB Prophet model',
                                                            'y':0.9,
                                                            'x':0.5,
                                                            'xanchor': 'center',
                                                            'yanchor': 'top'},
                                                     xaxis_title="Date", 
                                                     yaxis_title="Closing price in pounds (£)")
                           ),
                     className="six columns"),


            dcc.Graph(id='line-graph4', style={'display': 'inline-block', 'width':'50%'},
          figure=go.Figure(data = [go.Scatter(
                                              x = df['Date'],
                                              y = df['Close'],
                                              mode = 'lines',
                                              name = 'Observed',
                                              line={'color': 'orange'}),
                                   go.Scatter(
                                              x = df_future['ds'],
                                              y = df_future['yhat'],
                                              mode = 'lines',
                                              name = 'Predicted',
                                              line={'color': 'green'}),
                                   go.Scatter(
                                              x = df_future['ds'],
                                              y = df_future['yhat_lower'],
                                              mode = 'lines',
                                              fill = None,
                                              name = 'Lower range',
                                              line={'color': 'blue'}),
                                   go.Scatter(
                                              x = df_future['ds'],
                                              y = df_future['yhat_upper'],
                                              mode = 'lines',
                                              fill = 'tonexty',
                                              name = 'Upper range',
                                              line={'color': 'blue'}),],
                                    layout=go.Layout(title={'text':'Future Close prices Prediction for FB Prophet model',
                                                            'y':0.9,
                                                            'x':0.5,
                                                            'xanchor': 'center',
                                                            'yanchor': 'top'},
                                                     xaxis_title="Date", 
                                                     yaxis_title="Closing price in pounds (£)")
                          ),
                    className="six columns"),
            
                  ],
    className="six columns") 
    ])



app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/



INFO:__main__:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [ ]:
! pip install pyngrok

     |████████████████████████████████| 745 kB 1.3 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=18991 sha256=1f1a521b2e3f5e9a090219d9aac6c85bd86b552ea03f4b9cebc954eb5d9d34e9
  Stored in directory: /home/vamshi/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
! ngrok authtoken 27TEY85CerAWViwUlrnlfEI0KK2_5KQEpnFoGa6udBefZu81E


Authtoken saved to configuration file: /home/vamshi/.ngrok2/ngrok.yml


In [ ]:
from pyngrok import ngrok

# Open a HTTP tunnel on the default port 80
public_url = ngrok.connect(port = '8050')

INFO:pyngrok.ngrok:Opening tunnel named: http-80-5442be1f-97ce-4309-90a9-e6fa4ede6f2b
INFO:pyngrok.process.ngrok:t=2022-04-13T20:14:53+0100 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2022-04-13T20:14:53+0100 lvl=info msg="using configuration at default config path" path=/home/vamshi/.ngrok2/ngrok.yml
INFO:pyngrok.process.ngrok:t=2022-04-13T20:14:53+0100 lvl=info msg="open config file" path=/home/vamshi/.ngrok2/ngrok.yml err=nil
INFO:pyngrok.process.ngrok:t=2022-04-13T20:14:53+0100 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
INFO:pyngrok.process.ngrok:t=2022-04-13T20:14:54+0100 lvl=info msg="tunnel session started" obj=tunnels.session
INFO:pyngrok.process.ngrok:t=2022-04-13T20:14:54+0100 lvl=info msg="client session established" obj=csess id=c31648765f48
INFO:pyngrok.process.ngrok:t=2022-04-13T20:14:54+0100 lvl=info msg=start pg=/api/tunnels id=f6585edcf5bdd665
INFO:pyngrok.process.ngrok:t=2022-04-13T20:14:54+0100 lvl=info msg=end pg=

In [ ]:
public_url


<NgrokTunnel: "http://634f-86-22-15-144.ngrok.io" -> "http://localhost:80">

In [ ]:
ngrok.kill()

INFO:pyngrok.process:Killing ngrok process: 15459


In [ ]:
# from pandas import to_datetime
# from fbprophet import Prophet
# from matplotlib import pyplot
# from pandas import DataFrame

# df1 = pd.read_csv('LLOYDS_Daily.csv')
# df1.drop(df1.columns.difference(['Date','Close']), 1, inplace=True)
# df1.columns = ['ds', 'y']
# df1['ds']= to_datetime(df1['ds'])

# # define the model
# model = Prophet(daily_seasonality=True)
# # fit the model
# model.fit(df1)
# # define the period for which we want a prediction
# future = list()
# for i in range(3,13):
#     date = '2022-%02d' % i
#     future.append([date])
# future = DataFrame(future)
# future.columns = ['ds']
# future['ds']= to_datetime(future['ds'])
# # use the model to make a forecast
# forecast = model.predict(future)
# # summarize the forecast
# #print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])
# # plot forecast
# #model.plot(forecast)
# #pyplot.show()

# df1.set_index('ds', inplace=True)
# forecast.set_index('ds', inplace=True)
# viz_df = df1.join(forecast[['yhat', 'yhat_lower','yhat_upper']], how = 'outer')
# viz_df.head
# # df1.index = pd.to_datetime(df1.index)
# # connect_date = df1.index[-2]
# # mask = (forecast.index > connect_date)
# # predict_df = forecast.loc[mask]
# # viz_df = df1.join(predict_df[['yhat', 'yhat_lower','yhat_upper']], how = 'outer')
# # viz_df['yhat_scaled']=np.exp(viz_df['yhat'])

# # fig, ax1 = plt.subplots()
# # ax1.plot(viz_df.y)
# # ax1.plot(viz_df.yhat, color='black', linestyle=':')
# # ax1.fill_between(viz_df.index, np.exp(viz_df['yhat_upper']), np.exp(viz_df['yhat_lower']), alpha=0.5, color='darkgray')
# # ax1.set_ylabel('y')
# # ax1.set_xlabel('Date')


   


<bound method NDFrame.head of                     y       yhat  yhat_lower  yhat_upper
ds                                                      
2010-01-04  52.259998        NaN         NaN         NaN
2010-01-05  54.029999        NaN         NaN         NaN
2010-01-06  54.590000        NaN         NaN         NaN
2010-01-07  56.930000        NaN         NaN         NaN
2010-01-08  56.900002        NaN         NaN         NaN
...               ...        ...         ...         ...
2022-08-01        NaN  37.836275   29.418647   46.184347
2022-09-01        NaN  36.511986   27.965351   45.331872
2022-10-01        NaN  37.050779   26.942592   46.249000
2022-11-01        NaN  38.061117   26.934182   47.889781
2022-12-01        NaN  38.226626   26.845913   47.779852

[3093 rows x 4 columns]>